# 🔽 Extração e Preparação do Dataset `trn.json` do Google Drive
Este notebook realiza as seguintes etapas:
1. Monta O Drive, para salvamento
2. Instala o pacote `gdown` caso não esteja instalado;
3. Baixa um arquivo `.zip` do Google Drive;
4. Extrai apenas o arquivo `trn.json` de dentro do `.zip`;
5. Remove o `.zip` para economizar espaço em disco;
6. Converte o `trn.json` para o formato `.jsonl` com campos `prompt` e `completion`.


In [ ]:
## montar o google driver
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Instala o gdown automaticamente se necessário
import subprocess, sys

try:
    import gdown
except ImportError:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "gdown"])
    import gdown

## 📁 Baixando o ZIP do Google Drive
Vamos usar o `gdown` para baixar o arquivo `.zip` diretamente de um link público do Google Drive.


In [ ]:
DATA_DIR = "/content/drive/MyDrive/FIAP/tcFase3Grupo9"

In [ ]:
zip_file_id = "12zH4mL2RX8iSvH0VCNnd3QxO4DzuHWnK"
zip_output = f"{DATA_DIR}/dados.zip"

# Baixa o arquivo ZIP
gdown.download(f"https://drive.google.com/uc?id={zip_file_id}", zip_output, quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=12zH4mL2RX8iSvH0VCNnd3QxO4DzuHWnK
From (redirected): https://drive.google.com/uc?id=12zH4mL2RX8iSvH0VCNnd3QxO4DzuHWnK&confirm=t&uuid=bfdc03da-1465-418a-9f51-7f4fab620e38
To: /content/drive/MyDrive/FIAP/tcFase3Grupo9/dados.zip
100%|██████████| 890M/890M [00:20<00:00, 42.5MB/s]


'/content/drive/MyDrive/FIAP/tcFase3Grupo9/dados.zip'

## 📦 Extraindo apenas o `trn.json` do ZIP
Usamos o módulo `zipfile` da biblioteca padrão para extrair somente o arquivo desejado.


In [ ]:
import zipfile
import os
import shutil
import gzip

DATA_DIR = "/content/drive/MyDrive/FIAP/tcFase3Grupo9"
ZIP_NAME = "dados.zip"
texto_desejado = "trn.json"

zip_path = os.path.join(DATA_DIR, ZIP_NAME)
temp_extract_dir = os.path.join(DATA_DIR, "tmp_extracao")
os.makedirs(temp_extract_dir, exist_ok=True)

extraido = None
nome_final = None

# Etapa 1: Abrir e localizar arquivo com 'trn.json'
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    for arquivo in zip_ref.namelist():
        if texto_desejado in os.path.basename(arquivo):
            zip_ref.extract(arquivo, temp_extract_dir)
            caminho_extraido = os.path.join(temp_extract_dir, arquivo)
            nome_extraido = os.path.basename(arquivo)
            destino = os.path.join(DATA_DIR, nome_extraido)
            shutil.move(caminho_extraido, destino)
            extraido = destino
            print(f"✅ Arquivo extraído: {extraido}")
            break

# Etapa 2: Descompactar .gz se necessário
if extraido:
    if extraido.endswith('.gz'):
        nome_final = extraido.replace('.gz', '')
        with gzip.open(extraido, 'rb') as f_in, open(nome_final, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)
        os.remove(extraido)
        print(f"📂 Arquivo GZ descompactado para: {nome_final}")
    elif extraido.endswith('.json'):
        nome_final = extraido
        print(f"📄 Arquivo JSON pronto para uso: {nome_final}")
    else:
        print(f"⚠️ Extensão desconhecida: {extraido}")
else:
    print("❌ Nenhum arquivo contendo 'trn.json' foi encontrado.")

# Etapa 3: Limpeza da pasta temporária (sem tocar no conteúdo do Drive)
if os.path.exists(temp_extract_dir):
    try:
        shutil.rmtree(temp_extract_dir)
        print(f"🗑️ Pasta temporária removida: {temp_extract_dir}")
    except Exception as e:
        print(f"⚠️ Erro ao remover pasta temporária: {e}")

✅ Arquivo extraído: /content/drive/MyDrive/FIAP/tcFase3Grupo9/trn.json.gz
📂 Arquivo GZ descompactado para: /content/drive/MyDrive/FIAP/tcFase3Grupo9/trn.json
🗑️ Pasta temporária removida: /content/drive/MyDrive/FIAP/tcFase3Grupo9/tmp_extracao


## 📄 Função para preparar o JSONL
Transformamos o `trn.json` em um arquivo `.jsonl` contendo `prompt` e `completion`, ideal para fine-tuning de modelos de linguagem.


In [ ]:
import json
import re
import random
import html

def limpar_html(texto):
    # Remove tags HTML
    texto = re.sub(r'<[^>]+>', '', texto)
    # Converte entidades HTML para caracteres reais
    texto = html.unescape(texto)
    # Substitui espaços não separáveis por espaço normal
    texto = texto.replace('\u00A0', ' ')
    # Remove espaços/quebras extras
    texto = re.sub(r'\s+', ' ', texto).strip()
    return texto

def tem_palavras_coladas(texto):
    # Detecta padrão letra minúscula seguida de maiúscula sem espaço
    return bool(re.search(r'[a-z][A-Z]', texto))

def preparar_dataset_para_unsloth_instruction(caminho_entrada, caminho_saida, limite_linhas=100_000):
    frase_indesejada = "--This text refers to an out of print or unavailable edition of this title."

    midia_regex = re.compile(
        r"\b(bbc|cnn|bloomberg|reuters|forbes|washington post|nytimes|the new york times|"
        r"wall street journal|usa today|los angeles times|san francisco chronicle|"
        r"the boston globe|newsweek|time magazine|the atlantic|new yorker|slate|"
        r"the nation|the week)\b",
        re.IGNORECASE
    )

    instrucoes = [
        "You are a product seller. Generate a detailed product description based on the title.",
        "As a salesperson, write a product description for the following title.",
        "You're writing marketing content for a product. Describe the product with the given title.",
        "Imagine you're introducing this product to a customer. Create a product summary from this title.",
        "As a store assistant, explain what this product is based on the title.",
        "You're preparing a commercial listing. Write a compelling description using the title provided."
    ]

    cont_validas = 0
    cont_ignoradas = 0
    erros_json = 0
    titulos_vazios = 0
    descricoes_vazias = 0
    ignoradas_por_midia = 0
    ignoradas_por_colagem = 0
    ignoradas_por_titulo_igual_descricao = 0

    with open(caminho_entrada, 'r', encoding='utf-8') as f_in, \
         open(caminho_saida, 'w', encoding='utf-8') as f_out:

        for linha in f_in:
            if cont_validas >= limite_linhas:
                break

            try:
                data = json.loads(linha)
                titulo = data.get("title", "").strip()
                descricao = data.get("content", "").strip()

                if not titulo:
                    titulos_vazios += 1
                    cont_ignoradas += 1
                    continue

                if not descricao:
                    descricoes_vazias += 1
                    cont_ignoradas += 1
                    continue

                if midia_regex.search(descricao):
                    ignoradas_por_midia += 1
                    cont_ignoradas += 1
                    continue

                if tem_palavras_coladas(descricao):
                    ignoradas_por_colagem += 1
                    cont_ignoradas += 1
                    continue

                # Limpa e remove frase indesejada da descrição para comparar
                descricao_limpa = limpar_html(descricao.replace(frase_indesejada, "")).strip()

                # Ignora se título e descrição limpos forem iguais (case-insensitive)
                if titulo.lower() == descricao_limpa.lower():
                    ignoradas_por_titulo_igual_descricao += 1
                    cont_ignoradas += 1
                    continue

                entrada = {
                    "instruction": random.choice(instrucoes),
                    "input": titulo,
                    "output": descricao_limpa
                }

                f_out.write(json.dumps(entrada, ensure_ascii=False) + '\n')
                cont_validas += 1

            except json.JSONDecodeError:
                erros_json += 1
                cont_ignoradas += 1

    print("✅ Dataset gerado com sucesso!")
    print(f"📄 Linhas válidas: {cont_validas}")
    print(f"🚫 Linhas ignoradas: {cont_ignoradas}")
    print(f"   └─ 🧨 Erros de JSON: {erros_json}")
    print(f"   └─ ❌ Títulos vazios: {titulos_vazios}")
    print(f"   └─ ❌ Descrições vazias: {descricoes_vazias}")
    print(f"   └─ ❌ Título igual à descrição: {ignoradas_por_titulo_igual_descricao}")
    print(f"   └─ 📰 Ignoradas por mencionar veículos de mídia: {ignoradas_por_midia}")
    print(f"   └─ ❌ Ignoradas por conter palavras coladas: {ignoradas_por_colagem}")
    print(f"💾 Arquivo salvo em: {caminho_saida}")


## ▶️ Executando a Preparação dos Dados


In [ ]:
CAMINHO_JSON = f"{DATA_DIR}/trn.json"
SAIDA_JSONL = f"{DATA_DIR}/dataset_preparado_100_alpaca.jsonl"
# preparar_dataset_para_unsloth(CAMINHO_JSON, SAIDA_JSONL)
preparar_dataset_para_unsloth_instruction(CAMINHO_JSON, SAIDA_JSONL)

✅ Dataset gerado com sucesso!
📄 Linhas válidas: 100000
🚫 Linhas ignoradas: 172400
   └─ 🧨 Erros de JSON: 0
   └─ ❌ Títulos vazios: 20
   └─ ❌ Descrições vazias: 84366
   └─ ❌ Título igual à descrição: 22
   └─ 📰 Ignoradas por mencionar veículos de mídia: 9287
   └─ ❌ Ignoradas por conter palavras coladas: 78705
💾 Arquivo salvo em: /content/drive/MyDrive/FIAP/tcFase3Grupo9/dataset_preparado_100_alpaca.jsonl
